# Construction of the pMEC9001, 2 and 3 vectors

The vector pMEC1049 vector was used in [Romaní et al. 2014](http://www.sciencedirect.com/science/article/pii/S096085241401757X) The pMEC1049 expresses a D-xylose metabolic pathway and has a hygromycin selectable marker. The details of the construction of pMEC1049 can be found [here](pMEC1049.ipynb).

This document describe the construction of the pMEC9001, 2 and 3 vectors. The pMEC9001 is the pMEC1049 with an additional expression cassette for the Saccharomyces cerevisiae gene HAA1(YPR008W). The pMEC9002 is the pMEC1049 with an additional expression cassette for S. cerevisiae PRS3(YHL011C) and pMEC9003 has both of them.

| Vector   | Relevant property                                            |
|----------|--------------------------------------------------------------|
| pMEC9001 | [HAA1](http://www.yeastgenome.org/locus/S000006212/overview) |
| pMEC9002 | [PRS3](http://www.yeastgenome.org/locus/S000001003/overview) |
| pMEC9003 | HAA1 & PRS3                                                  |

Normally, this would be done following the yeast pathway kit strategy by adding genes with a set of new promoters and terminators, but in this case a requirement was to retain the native promoters and terminators for HAA1 and PRS3.

The strategy involves linearizing the vector in two locations (before and after the xylose pathway) and adding the HAA1 and PRS3 expression cassettes amplified using tailed primers.


## The PRS3 construct

The PRS3 cassette was previously cloned according to the description below in vector YEpJCP according to this description:

"obtained by PCR amplification of fragment carrying PRS3 from Saccharomyces cerevisiae CEN.PK113-7D genomic DNA using appropriate primers and insertion into plasmid pGEM-T Easy. Cloning into YEplac195KanMX using EcoRI digestion sites." [Cunha et al. 2015](http://www.sciencedirect.com/science/article/pii/S0960852415006707)

Primers:

    P1: TTATCTTCATCACCGCCATAC
    P2: ACAAGAGAAACTTTTGGGTAAAATG

The exact same PRS3 fragment will be cloned in pMEC9002.

In [65]:
import pydna

In [66]:
p1,p2 = pydna.parse_primers('''
>P1
TTATCTTCATCACCGCCATAC
>P2
ACAAGAGAAACTTTTGGGTAAAATG
''')

We gain access to the S. cerevisiae genome through the [pygenome](https://pypi.python.org/pypi/pygenome) module.

In [67]:
from pygenome import sg

In [68]:
PRS3_locus = pydna.Dseqrecord(sg.gene["PRS3"].locus())

The PRS3_locus contain the DNA from the end of the upstream ORF to the beginning of the downstream ORF.

In [69]:
PRS3_locus

Dseqrecord(-2963)

In [70]:
PRS3_product = pydna.pcr(p1, p2, PRS3_locus)

In [71]:
PRS3_product

Amplicon(1546)

In [72]:
PRS3_product.figure()

5TTATCTTCATCACCGCCATAC...CATTTTACCCAAAAGTTTCTCTTGT3
                         ||||||||||||||||||||||||| tm 55.7 (dbd) 63.4
                        3GTAAAATGGGTTTTCAAAGAGAACA5
5TTATCTTCATCACCGCCATAC3
 ||||||||||||||||||||| tm 55.5 (dbd) 62.9
3AATAGAAGTAGTGGCGGTATG...GTAAAATGGGTTTTCAAAGAGAACA5

The primers anneal perfectly to the template, so this is the PCR product we want.

# HAA1 construct

We will now do the same with the HAA1 cassette.

Vector: BHUM1737

Construction: obtained by PCR amplification of a SalI/BamHI fragment carrying HAA1 from yeast genomic DNA using appropriate primers and subsequent insertion into plasmid YEplac195.

Primers described in [Malcher et al. 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3063667/) supporting information, Table S1

    HAA1hc_fw: GTC GAC CCC ATT TCC CCT TTC TTT TCC
    HAA1hc_rev: GGA TCC ATA CCT CAT CTC TGC GTG TTC G

In [73]:
h1,h2 = pydna.parse('''
>HAA1hc_fw
GTC GAC CCC ATT TCC CCT TTC TTT TCC
>HAA1hc_rev
GGA TCC ATA CCT CAT CTC TGC GTG TTC G
''')

In [74]:
HAA1_locus=pydna.Dseqrecord(sg.gene["HAA1"].locus())

In [75]:
HAA1_locus

Dseqrecord(-4085)

In [76]:
HAA1_product = pydna.pcr(h1, h2, HAA1_locus)

In [77]:
HAA1_product

Amplicon(3227)

In [78]:
HAA1_product.figure()

      5CCCATTTCCCCTTTCTTTTCC...CGAACACGCAGAGATGAGGTAT3
                               |||||||||||||||||||||| tm 59.9 (dbd) 66.7
                              3GCTTGTGCGTCTCTACTCCATACCTAGG5
5GTCGACCCCATTTCCCCTTTCTTTTCC3
       ||||||||||||||||||||| tm 57.0 (dbd) 67.6
      3GGGTAAAGGGGAAAGAAAAGG...GCTTGTGCGTCTCTACTCCATA5

These primers are tailed, but we have no reason to include these tails (containing restriction sites). We therefore cut six bp from the beginning and six bp from the end of the sequence:

In [79]:
HAA1_product = HAA1_product[6:-6]

In [80]:
HAA1_product.seq

Dseq(-3215)
CCCA..GTAT
GGGT..CATA

Now we will have to design tailed primers for the HAA1_product and the PRS3_product sequences so that we can add them to pMEC1049 by gap repair. First we have to decide with which restriction enzymes we should open the pMEC1049 vector.

The restriction enzymes below are candidates for linearizing the pMEC1049 before an after the cassette.

[XhoI](http://rebase.neb.com/rebase/enz/XhoI.html) [AleI](http://rebase.neb.com/rebase/enz/AleI.html) [OliI](http://rebase.neb.com/rebase/enz/OliI.html)

These enzymes are also unique in the pYPK0 based vectors, so we can use tha same strategy to create vectors expressing only the pRS3 and/or HAA1 but without the xylose pathway if needed.

In [81]:
pMEC1049 = pydna.read("pMEC1049.gb")

In [82]:
pMEC1049

File(id?)(o15488)

In [83]:
from Bio.Restriction import XhoI, AleI, OliI

In [84]:
pMEC1049_xho = pMEC1049.linearize(XhoI)

We design gap repair primers using the pydna assembly primers function

In [85]:
Hfw, Hrv = pydna.assembly_primers( (HAA1_product,), vector = pMEC1049_xho).pop()

In [86]:
Hfw.id = "Hfw"
Hrv.id = "Hrv"

In [87]:
Hfw = Hfw[1:]
Hrv = Hrv[:-1]

In [88]:
print( Hfw.format("tab") )

Hfw	GGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT



In [89]:
print( Hrv.format("tab") )

Hrv	AGACAAACCGTGGGACGAATTCTTAAGATGCTCGAATACCTCATCTCTGCG



In [90]:
HAA1_recombination_product = pydna.pcr(Hfw, Hrv, HAA1_locus)

In [91]:
HAA1_recombination_product

Amplicon(3284)

In [92]:
HAA1_recombination_product.figure()

                                  5CCCATTTCCCCTTTCT...CGCAGAGATGAGGTATT3
                                                      ||||||||||||||||| tm 51.5 (dbd) 55.2
                                                     3GCGTCTCTACTCCATAAGCTCGTAGAATTCTTAAGCAGGGTGCCAAACAGA5
5GGTTTTACCGTGTGCGGAGATCAGGTTCTGATCCCCCATTTCCCCTTTCT3
                                   |||||||||||||||| tm 51.1 (dbd) 59.1
                                  3GGGTAAAGGGGAAAGA...GCGTCTCTACTCCATAA5

In [93]:
asm_haa1 = pydna.Assembly((pMEC1049_xho, HAA1_recombination_product))

In [94]:
asm_haa1

Assembly:
Sequences........................: [15492] [3284]
Sequences with shared homologies.: [15492] [3284]
Homology limit (bp)..............: 25
Number of overlaps...............: 2
Nodes in graph(incl. 5' & 3')....: 4
Only terminal overlaps...........: No
Circular products................: [18703]
Linear products..................: [18741] [18738] [38] [35]

In [95]:
candidate = asm_haa1.circular_products[0]

In [96]:
candidate.figure()

 -|pMEC1049|34
|           \/
|           /\
|           34|3284bp_PCR_prod|35
|                              \/
|                              /\
|                              35-
|                                 |
 ---------------------------------

In [97]:
pMEC9001 = candidate.synced(pMEC1049)

In [98]:
pMEC9001.stamp()

cSEGUID_BgBOlNv8lWGLExKGOpZa8J83H9I

In [99]:
pMEC9001.locus="pMEC9001"

The pMEC9001 is the pMEC1049 with HAA1. The sequence can be downloaded using the link below.

In [100]:
pMEC9001.write("pMEC9001.gb")

## PRS3

We will now make a pMEC1049 with PRS3 called pMEC9002.

In [101]:
pMEC1049_oli = pMEC1049.linearize(OliI)

The integration site was chosen to be the uniqie OliI site.

In [102]:
Pfw, Prv = pydna.assembly_primers((PRS3_product,), vector=pMEC1049_oli).pop()

In [103]:
Pfw.id = "Pfw"
Prv.id = "Prv"

In [104]:
Prv=Prv[:-2]

In [106]:
print( Pfw.format("tab"))
print( Prv.format("tab"))

Pfw	TAACGATGTAGTACAGCGTTTCCGCTTTTTCACCCTTATCTTCATCACCGC

Prv	CATAAGTACCCATCCAAGAGCACGCTTATTCACCAACAAGAGAAACTTTTG



In [107]:
PRS3_recombination_product = pydna.pcr(Pfw, Prv, PRS3_locus)

In [108]:
PRS3_recombination_product

Amplicon(1616)

The recombination was designed for OliI but AleI was used.

In [109]:
pMEC1049_ale = pMEC1049.linearize(AleI)

In [110]:
asm_prs3 = pydna.Assembly((pMEC1049_ale, PRS3_recombination_product))

In [111]:
asm_prs3

Assembly:
Sequences........................: [15488] [1616]
Sequences with shared homologies.: [15488] [1616]
Homology limit (bp)..............: 25
Number of overlaps...............: 2
Nodes in graph(incl. 5' & 3')....: 4
Only terminal overlaps...........: No
Circular products................: [17034]
Linear products..................: [17069] [17069] [35] [35]

In [112]:
candidate = asm_prs3.circular_products[0]

In [113]:
candidate

Contig(o17034)

In [114]:
pMEC9002 = candidate.synced(pMEC1049)

In [115]:
pMEC9002.locus = "pMEC9002"

In [116]:
pMEC9002.stamp()

cSEGUID_5qldWd2jGxUyMnMKECLGpNvJv2M

The pMEC9002 vector is the pMEC1049 with PRS3

In [118]:
pMEC9002.write("pMEC9002.gb")

## pMEC9003

The HAA1 and PRS3 cassettes were added to the plasmid in one step to the plasmid digested with both XhoI and AleI. Cutting with XhoI and AleI makes two fragments about 6 and 9 kb.

In [119]:
pMEC1049_6kb, pMEC1049_9kbp = pMEC1049.cut(XhoI, AleI)

In [120]:
pMEC1049_6kb

Dseqrecord(-6017)

In [121]:
pMEC1049_9kbp

Dseqrecord(-9475)

In [122]:
pMEC1049_6kb.locus = "pMEC1049_6kb"
pMEC1049_9kbp.locus = "pMEC1049_9kbp"

In [123]:
asm_prs_haa = pydna.Assembly((pMEC1049_6kb, pMEC1049_9kbp, HAA1_recombination_product, PRS3_recombination_product))

In [124]:
asm_prs_haa

Assembly:
Sequences........................: [6017] [9475] [3284] [1616]
Sequences with shared homologies.: [6017] [3284] [1616] [9475]
Homology limit (bp)..............: 25
Number of overlaps...............: 4
Nodes in graph(incl. 5' & 3')....: 6
Only terminal overlaps...........: No
Circular products................: [20249]
Linear products..................: [20287] [20284] [20284] [20284] [18703] [17038] [14305] [12724] [11056] [10844] [9263] [7598] [38] [35] [35] [35]

In [125]:
candidate = asm_prs_haa.circular_products[0]

In [126]:
candidate

Contig(o20249)

In [127]:
pMEC9003 = candidate.synced(pMEC1049)

In [129]:
pMEC9003.stamp()

cSEGUID_hhymwO1hS1IXp9n4XX4eODGWuhU

In [130]:
pMEC9003.locus="pMEC9003"

pMEC9003 is the pMEC1049 with both HAA1 and PRS3. The sequence can be downloaded from the link below.

In [131]:
pMEC9003

Dseqrecord(o20249)

In [132]:
pMEC9003.write("pMEC9003.gb")